# Setup

In [0]:
files_arrival_path = "dbfs:/FileStore/Northwind/Raw/"
database_name = "northwind"
file_format = "CSV"
files = [
    "categories.csv",
    "customer_customer_demo.csv",
    "customer_demographics.csv",
    "customers.csv",
    "employee_territories.csv",
    "employees.csv",
    "order_details.csv",
    "orders.csv",
    "products.csv",
    "regions.csv",
    "shippers.csv",
    "suppliers.csv",
    "territories.csv",
    "us_states.csv"
]

**Community Edition Limitation**

Remains from files and tables that do not get cleaned. Comment/uncomment as necessary.

In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/northwind.db", recurse=True)
spark.sql(f"DROP SCHEMA IF EXISTS {database_name} CASCADE")

Out[6]: True

# Creating Database

In [0]:
spark.sql(f"""
    CREATE SCHEMA IF NOT EXISTS {database_name}
    COMMENT 'Northwind delta lakehouse';
""")

spark.sql(f"USE SCHEMA {database_name};")

Out[3]: DataFrame[]

# Creating tables

In [0]:
for file in files:
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS br_{file.split('.')[0]}
        TBLPROPERTIES('quality'='bronze')
    """)

    spark.sql(f"""
        COPY INTO br_{file.split('.')[0]}
        FROM '{files_arrival_path+file}'
        FILEFORMAT = {file_format}
        FORMAT_OPTIONS ('mergeSchema'='true', 'header'='true')
        COPY_OPTIONS ('mergeSchema'='true');
    """)